In [2]:
import nibabel as nib

nii = nib.load('../BraTS2021_00621_flair.nii.gz').get_fdata() 
print(nii.shape) # (256,256,256)

(240, 240, 155)


In [18]:
label = nib.load('../BraTS2021_00495_seg.nii.gz').get_fdata() 
output_dir = './seg_result2'
os.makedirs(output_dir, exist_ok=True)
label = np.transpose(label, (2, 0, 1))

for idx, slice_2d in enumerate(label):
    print(slice_2d.max())
    cv2.imwrite(f'{output_dir}/slice_{idx:03d}.png', slice_2d*64)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [13]:
slice_2d.max()

0.0

In [17]:
import torch
import pickle
import cv2
import numpy as np
import os

from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox



nii_file = '../BraTS2021_00495_flair.nii.gz'
nii = nib.load(nii_file).get_fdata()  # shape: (H, W, D) -> (240, 240, 155)
nii = np.transpose(nii, (2, 0, 1))    # shape: (D, H, W) for iteration

# 2. Load model
with open('./brainMRI_model_weights.pkl', 'rb') as f:
    model = pickle.load(f)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


output_dir = './flair_result2'
os.makedirs(output_dir, exist_ok=True)

# 3. Iterate through slices
for idx, slice_2d in enumerate(nii):
    img0 = cv2.normalize(slice_2d, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    img0 = cv2.cvtColor(img0, cv2.COLOR_GRAY2BGR)  # YOLO expects 3 channels

    # Resize and prepare for model
    img = letterbox(img0, new_shape=640)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR->RGB->CHW
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).float().to(device) / 255.0
    img = img.unsqueeze(0)

    # Inference
    with torch.no_grad():
        pred = model(img)[0]
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

    # Draw detections
    if len(pred) > 0 :
        print(pred, idx)
    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in det:
                xyxy = [int(x.item()) for x in xyxy]
                label = f'{int(cls)} {conf:.2f}'
                cv2.rectangle(img0, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
                cv2.putText(img0, label, (xyxy[0], xyxy[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)

    # Show or save result
    # 내부 루프에서 이미지 저장
    cv2.imwrite(f'{output_dir}/slice_{idx:03d}.png', img0)

    # cv2.imshow(f'Slice {idx}', img0)
    key = cv2.waitKey(1)
    if key == ord('q'): break

    




[tensor([], device='cuda:0', size=(0, 6))] 0
[tensor([], device='cuda:0', size=(0, 6))] 1
[tensor([], device='cuda:0', size=(0, 6))] 2
[tensor([], device='cuda:0', size=(0, 6))] 3
[tensor([], device='cuda:0', size=(0, 6))] 4
[tensor([], device='cuda:0', size=(0, 6))] 5
[tensor([], device='cuda:0', size=(0, 6))] 6
[tensor([], device='cuda:0', size=(0, 6))] 7
[tensor([], device='cuda:0', size=(0, 6))] 8
[tensor([], device='cuda:0', size=(0, 6))] 9
[tensor([], device='cuda:0', size=(0, 6))] 10
[tensor([], device='cuda:0', size=(0, 6))] 11
[tensor([], device='cuda:0', size=(0, 6))] 12
[tensor([], device='cuda:0', size=(0, 6))] 13
[tensor([], device='cuda:0', size=(0, 6))] 14
[tensor([], device='cuda:0', size=(0, 6))] 15
[tensor([], device='cuda:0', size=(0, 6))] 16
[tensor([], device='cuda:0', size=(0, 6))] 17
[tensor([], device='cuda:0', size=(0, 6))] 18
[tensor([], device='cuda:0', size=(0, 6))] 19
[tensor([], device='cuda:0', size=(0, 6))] 20
[tensor([], device='cuda:0', size=(0, 6))] 2